In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime


In [2]:
file_csv = '~/code/amandamor/output/combined_flat2_csv.csv'
df_raw_data = pd.read_csv(file_csv)

/var/folders/bn/g48wyj9119lb2ytw0l5thk840000gn/T/ipykernel_14142/1460808056.py:2: DtypeWarning: Columns (22,53,62,71,80,89,98,107,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw_data = pd.read_csv(file_csv)


In [3]:
import pipeline_cleaning

In [4]:
clean_data = pipeline_cleaning.clean_data(df_raw_data)
clean_data

/Users/amandamor/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['failed_to_finish_reason'] = df['failed_to_finish_reason'].apply(lambda x: 0 if pd.isna(x) else 1)
/Users/amandamor/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['margin'] = df.apply(lambda row: row['distance'] if pd.isna(row['margin']) and (row['win_or_lose'] == 1 or row['failed_to_finish_reason'] == 1) else row['m

,date,event_number,distance,race_class,race_class_normalised,total_prize_money,track_condition,race_type,track_type,barrier,...,jockey_id,tainer_id,dslr,official rating,wfa,weight_adjustment,betfair_starting_price,pre_race_master_rating_int,starting_price,current_age
0,2020-07-20,1,1100,6,Handicap,4231,GOOD TO FIRM,FLAT,TURF,8,...,4020,3897,22.0,65.0,0.0,7.0,4.383864,75.0,4.0,5.0
1,2020-07-20,7,1760,6,Handicap,4231,GOOD TO FIRM,FLAT,TURF,6,...,4320,3985,22.0,61.0,0.0,11.0,4.666511,75.0,4.2,4.0
2,2020-07-20,7,1760,6,Handicap,4231,GOOD TO FIRM,FLAT,TURF,4,...,4021,3836,10.0,63.0,0.0,12.0,21.818160,63.0,19.0,4.0
3,2020-07-20,7,1760,6,Handicap,4231,GOOD TO FIRM,FLAT,TURF,7,...,3944,1616,7.0,65.0,0.0,10.0,4.211919,77.0,4.0,3.0
4,2020-07-20,7,1760,6,Handicap,4231,GOOD TO FIRM,FLAT,TURF,2,...,3858,3836,22.0,65.0,0.0,7.0,25.454520,32.0,23.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282001,2023-09-28,3,1320,2,Handicap,46920,GOOD TO FIRM,FLAT,TURF,6,...,4313,3897,33.0,100.0,0.0,2.0,7.091693,103.0,7.0,6.0
282002,2023-09-28,2,1760,4,Handicap,14296,GOOD TO FIRM,FLAT,TURF,9,...,1194,4552,9.0,71.0,0.0,19.0,24.998494,87.0,19.0,6.0
282003,2023-09-28,2,1760,4,Handicap,14296,GOOD TO FIRM,FLAT,TURF,4,...,4611,3773,22.0,74.0,0.0,16.0,15.000000,97.0,12.0,7.0
282004,2023-09-28,3,1320,2,Handicap,46920,GOOD TO FIRM,FLAT,TURF,8,...,875,1863,20.0,80.0,2.0,24.0,92.878897,95.0,67.0,7.0


In [6]:
pipeline_cleaning.transforming_data(clean_data)[0]

/Users/amandamor/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/Users/amandamor/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['jockey_id', 'tainer_id', 'margin', 'finish_position', 'event_number'], axis=1, inplace=True) # for now
/Users/amandamor/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,barrier_1,barrier_2,barrier_3,barrier_4,barrier_5,barrier_6,barrier_7,barrier_8,barrier_9,barrier_10,...,dslr,official rating,wfa,weight_adjustment,betfair_starting_price,pre_race_master_rating_int,starting_price,current_age,win_or_lose,gear
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.341000,-0.277886,-0.339061,-0.712174,-0.372942,-0.384102,-0.581721,-0.023006,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.341000,-0.530630,-0.339061,-0.071595,-0.368995,-0.384102,-0.573259,-0.353048,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.548722,-0.404258,-0.339061,0.088550,-0.129512,-1.090881,0.052875,-0.353048,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.600653,-0.277886,-0.339061,-0.231739,-0.375343,-0.266306,-0.581721,-0.683090,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.341000,-0.277886,-0.339061,-0.712174,-0.078739,-2.916727,0.222101,-0.023006,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129639,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.496792,0.859464,-0.339061,0.088550,-0.388832,1.088354,-0.624027,0.967119,0.0,0.0
129640,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.410241,0.859464,-0.339061,0.088550,-0.266600,0.676067,-0.370189,0.967119,0.0,0.0
129641,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.479482,0.922650,-0.339061,-0.071595,-0.099048,0.793863,0.052875,0.967119,0.0,1.0
129642,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.496792,0.859464,-0.339061,0.088550,-0.261739,1.147253,-0.327882,1.297161,0.0,0.0


In [49]:
clean_data['date'] = pd.to_datetime(clean_data['date'])
clean_data.drop(columns=['jockey_id', 'tainer_id', 'margin', 'finish_position', 'event_number'], axis=1, inplace=True)
clean_data.dropna(inplace=True) #instead of imputer
df_train = clean_data[(clean_data['date'].dt.year != 2022) & (clean_data['date'].dt.year != 2023)]
df_val = clean_data[clean_data['date'].dt.year == 2022]
df_test = clean_data[clean_data['date'].dt.year == 2023]
df_train.drop(columns=['date'], axis=1, inplace=True)
df_val.drop(columns=['date'], axis=1, inplace=True)
df_test.drop(columns=['date'], axis=1, inplace=True)

categorical_col = ['barrier', 'track_condition', 'race_type', 'track_type',
                    'race_class_normalised', 'race_class']
num_col = ['distance', 'total_prize_money', 'jockey_allowance',
            'handicap_weight', 'dslr', 'official rating', 'wfa',
            'weight_adjustment', 'betfair_starting_price',
            'pre_race_master_rating_int', 'starting_price', 'current_age']


/var/folders/bn/g48wyj9119lb2ytw0l5thk840000gn/T/ipykernel_74343/578957931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['date'] = pd.to_datetime(clean_data['date'])
/var/folders/bn/g48wyj9119lb2ytw0l5thk840000gn/T/ipykernel_74343/578957931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data.drop(columns=['jockey_id', 'tainer_id', 'margin', 'finish_position', 'event_number'], axis=1, inplace=True)
/var/folders/bn/g48wyj9119lb2ytw0l5thk840000gn/T/ipykernel_74343/578957931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a c

In [52]:
[col for col in df_train.columns if col not in (num_col+categorical_col)]

['win_or_lose', 'gear']

In [66]:
df_test

,distance,race_class,race_class_normalised,total_prize_money,track_condition,race_type,track_type,barrier,win_or_lose,gear,jockey_allowance,handicap_weight,dslr,official rating,wfa,weight_adjustment,betfair_starting_price,pre_race_master_rating_int,starting_price,current_age
233788,1094,5,Handicap,7624,STANDARD,FLAT,AWT,7,0.0,1,0.0,130.0,10.0,65.0,0.0,10.0,8.000000,83.0,6.5,9.0
233789,1094,5,Handicap,7624,STANDARD,FLAT,AWT,8,0.0,1,0.0,131.0,45.0,66.0,0.0,9.0,8.800000,80.0,7.5,9.0
233790,1094,2,Handicap,23647,STANDARD,FLAT,AWT,1,0.0,1,0.0,122.0,53.0,90.0,0.0,18.0,14.877944,113.0,13.0,10.0
233791,1094,2,Handicap,23647,STANDARD,FLAT,AWT,3,0.0,0,0.0,127.0,108.0,95.0,0.0,13.0,5.900000,112.0,5.0,11.0
233792,1094,2,Handicap,23647,STANDARD,FLAT,AWT,5,0.0,0,0.0,129.0,14.0,97.0,0.0,11.0,29.958586,104.0,23.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282000,1320,2,Handicap,46920,GOOD TO FIRM,FLAT,TURF,2,1.0,1,0.0,134.0,7.0,98.0,2.0,6.0,8.723425,75.0,7.5,7.0
282001,1320,2,Handicap,46920,GOOD TO FIRM,FLAT,TURF,6,0.0,1,3.0,138.0,33.0,100.0,0.0,2.0,7.091693,103.0,7.0,6.0
282002,1760,4,Handicap,14296,GOOD TO FIRM,FLAT,TURF,9,0.0,0,0.0,121.0,9.0,71.0,0.0,19.0,24.998494,87.0,19.0,6.0
282003,1760,4,Handicap,14296,GOOD TO FIRM,FLAT,TURF,4,0.0,0,0.0,124.0,22.0,74.0,0.0,16.0,15.000000,97.0,12.0,7.0


In [23]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [71]:
categorical_preprocessor = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])
numerical_preprocessor = Pipeline([
    ('scaler', StandardScaler())
])
pipeline = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_col),
    ('numerical', numerical_preprocessor, num_col)
], remainder="passthrough", sparse_threshold=0)
pipeline

ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('categorical',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['barrier', 'track_condition', 'race_type',
                                  'track_type', 'race_class_normalised',
                                  'race_class']),
                                ('numerical',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['distance', 'total_prize_money',
                                  'jockey_allowance', 'handicap_weight', 'dslr',
                                  'official rating', 'wfa', 'weight_adjustment',
                                  'betfair_starting_price',
                                  'pre_race_master_rating_int',
                                  'starting_price', 'current_age'])])

In [79]:
pipeline.fit(df_train)
df_train_transformed = pipeline.transform(df_train)
df_val_transformed = pipeline.transform(df_val)
df_test_transformed = pipeline.transform(df_test)

/Users/amandamor/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [80]:
pipeline.transform(df_train)

array([[ 0.        ,  0.        ,  0.        , ..., -0.02300598,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.3530478 ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.3530478 ,
         0.        ,  0.        ],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.96711945,
         0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.        , ...,  1.29716126,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  1.95724489,
         0.        ,  0.        ]])

In [81]:
# pipeline.get_feature_names_out()


In [82]:
categorical_feature_names = pipeline.named_transformers_['categorical'].named_steps['onehot'].get_feature_names_out(input_features=categorical_col)

# Obter os nomes das colunas numéricas
numerical_feature_names = num_col
remainder_col_names = [col for col in df_train.columns if col not in (num_col+categorical_col)]

# Combinar os nomes das colunas categóricas e numéricas
all_feature_names = list(categorical_feature_names) + numerical_feature_names + remainder_col_names
all_feature_names

['barrier_1',
 'barrier_2',
 'barrier_3',
 'barrier_4',
 'barrier_5',
 'barrier_6',
 'barrier_7',
 'barrier_8',
 'barrier_9',
 'barrier_10',
 'barrier_11',
 'barrier_12',
 'barrier_13',
 'barrier_14',
 'barrier_15',
 'barrier_16',
 'barrier_17',
 'barrier_18',
 'barrier_19',
 'barrier_20',
 'track_condition_FAST',
 'track_condition_FIRM',
 'track_condition_GOOD',
 'track_condition_GOOD TO FIRM',
 'track_condition_GOOD TO SOFT',
 'track_condition_HEAVY',
 'track_condition_SOFT',
 'track_condition_STANDARD',
 'track_condition_STANDARD TO FAST',
 'track_condition_STANDARD TO SLOW',
 'race_type_NATIONAL_HUNT_FLAT',
 'track_type_TURF',
 'race_class_normalised_Claiming',
 'race_class_normalised_Conditions',
 'race_class_normalised_Grade 1',
 'race_class_normalised_Grade 2',
 'race_class_normalised_Group 1',
 'race_class_normalised_Group 2',
 'race_class_normalised_Group 3',
 'race_class_normalised_Handicap',
 'race_class_normalised_Listed',
 'race_class_normalised_Maiden',
 'race_class_norma

In [83]:
df_test_transformed_with_columns = pd.DataFrame(df_test_transformed, columns=all_feature_names)
df_val_transformed_with_columns = pd.DataFrame(df_val_transformed, columns=all_feature_names)
df_train_transformed_with_columns = pd.DataFrame(df_train_transformed, columns=all_feature_names)

In [84]:
df_train_transformed_with_columns

,barrier_1,barrier_2,barrier_3,barrier_4,barrier_5,barrier_6,barrier_7,barrier_8,barrier_9,barrier_10,...,dslr,official rating,wfa,weight_adjustment,betfair_starting_price,pre_race_master_rating_int,starting_price,current_age,win_or_lose,gear
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.341000,-0.277886,-0.339061,-0.712174,-0.372942,-0.384102,-0.581721,-0.023006,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.341000,-0.530630,-0.339061,-0.071595,-0.368995,-0.384102,-0.573259,-0.353048,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.548722,-0.404258,-0.339061,0.088550,-0.129512,-1.090881,0.052875,-0.353048,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.600653,-0.277886,-0.339061,-0.231739,-0.375343,-0.266306,-0.581721,-0.683090,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.341000,-0.277886,-0.339061,-0.712174,-0.078739,-2.916727,0.222101,-0.023006,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129639,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.496792,0.859464,-0.339061,0.088550,-0.388832,1.088354,-0.624027,0.967119,0.0,0.0
129640,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.410241,0.859464,-0.339061,0.088550,-0.266600,0.676067,-0.370189,0.967119,0.0,0.0
129641,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.479482,0.922650,-0.339061,-0.071595,-0.099048,0.793863,0.052875,0.967119,0.0,1.0
129642,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.496792,0.859464,-0.339061,0.088550,-0.261739,1.147253,-0.327882,1.297161,0.0,0.0
